## Testing my pipeline

In [4]:
%load_ext autoreload
%autoreload 2

In [66]:
!python main.py --pdf_path '/Users/tiwaabruks/Desktop/Grad School/spring_2025/digital_texts_2/ocr-digitzation-project/input/mission-from-cape-coast-castle-full.pdf'  --book_name 'Mission From CCC'
               


✅ Uploaded mission-from-ccc to s3://book-digitization/mission-from-ccc/input/full_book.pdf


In [13]:
from dotenv import load_dotenv
import os
load_dotenv()

bucket = 'book-digitization'

In [31]:
test_event = {
    "Records": [
        {
            "s3": {
                "bucket": {"name": bucket},
                "object": {"key": "mission-from-ccc/input/full_book.pdf"}
            }
        }
    ]
}


In [32]:
from src.lambda_function import lambda_handler

response = lambda_handler(test_event, None)
print(response)


Received event: {"Records": [{"s3": {"bucket": {"name": "book-digitization"}, "object": {"key": "mission-from-ccc/input/full_book.pdf"}}}]}
This is the bucket,  book-digitization
This is the s3 key:  mission-from-ccc/input/full_book.pdf
ECS task started: arn:aws:ecs:us-east-1:322793536920:task/book-digitization-cluster/8683de4ae0674a2c8629740cec145407
{'statusCode': 200, 'body': '"ECS task started successfully"'}


In [ ]:
# !aws logs describe-log-streams --log-group-name "/aws/lambda/bookDigitizationLambda" --order-by LastEventTime --descending

# !aws logs get-log-events --log-group-name "/aws/lambda/bookDigitizationLambda" --log-stream-name "2025/05/31/[\$LATEST]e750e907705b48d8a860315430a497dc"

# !aws ecs describe-tasks --cluster book-digitization-cluster --tasks 023e6e34945c426885771dec70928ae0

# !aws logs get-log-events --log-group-name "/ecs/book-digitization" --log-stream-name "ecs/book-digitization-container/023e6e34945c426885771dec70928ae0"

^C


In [78]:
# !aws logs describe-log-streams --log-group-name "/ecs/book-digitization" --order-by LastEventTime --descending

# !aws logs get-log-events --log-group-name "/ecs/book-digitization" --log-stream-name "ecs/book-digitization-container/f3329dac90cc432f8adde2819b9cf92c"

# !aws logs describe-log-streams --log-group-name "/ecs/book-digitization" --order-by LastEventTime --descending | head -5

# !aws ecs describe-tasks --cluster book-digitization-cluster --tasks b934ee7b67f84aec8c5995c95b82c434

!aws logs get-log-events --log-group-name "/ecs/book-digitization" --log-stream-name "ecs/book-digitization-container/b934ee7b67f84aec8c5995c95b82c434"


{
    "events": [
        {
            "timestamp": 1748673915515,
            "message": "INFO:__main__:Starting digitization task for bucket: book-digitization, key: mission-from-ccc/input/full_book.pdf",
            "ingestionTime": 1748673919663
        },
        {
            "timestamp": 1748673915676,
            "message": "INFO:__main__:Downloading PDF to /tmp/full_book.pdf",
            "ingestionTime": 1748673919663
        },
        {
            "timestamp": 1748673916675,
            "message": "INFO:__main__:Initializing BookDigitizer for book: mission-from-ccc",
            "ingestionTime": 1748673919663
        },
        {
            "timestamp": 1748673916681,
            "message": "INFO:__main__:Running batch_and_upload_pdf with batch_size=20",
            "ingestionTime": 1748673919663
        },
        {
            "timestamp": 1748673920065,
            "message": "INFO:__main__:Batch upload complete",
            "ingestionTime": 1748673924599
        },


In [77]:
!docker buildx build --platform linux/amd64 -t book-worker .
!docker tag book-worker 322793536920.dkr.ecr.us-east-1.amazonaws.com/book-digitization:latest
!docker push 322793536920.dkr.ecr.us-east-1.amazonaws.com/book-digitization:latest

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 318B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.9-slim         0.2s
[+] Building 0.4s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 318B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.9-slim         0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfi

## Check that Bedrock works

In [53]:
from pathlib import Path
test_prompt = Path('src/parse_prompt.txt').read_text()

In [54]:
sample_text = Path('/Users/tiwaabruks/Downloads/full_book-batch-9.txt').read_text()

In [55]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')
test_prompt = "Return this JSON: {'test': 'success'}"

response = bedrock.invoke_model(
    modelId='anthropic.claude-3-sonnet-20240229-v1:0',   
    body=json.dumps({
        "anthropic_version": "bedrock-2023-05-31", 
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": test_prompt}]
    })
)

result = json.loads(response['body'].read())
print(result)

{'id': 'msg_bdrk_014Grjfph2462sjfisFSvzZs', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'text', 'text': "{'test': 'success'}"}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 18, 'output_tokens': 10}}


In [60]:
with open('src/parse_prompt.txt', 'r') as f:
    prompt = f.read()

with open('/Users/tiwaabruks/Downloads/full_book-batch-9.txt', 'r') as f:
    sample_text = f.read()

response = bedrock.invoke_model(
    modelId='anthropic.claude-3-sonnet-20240229-v1:0',
    body=json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4000,
        "messages": [{"role": "user", "content": f"{prompt}\n\n{sample_text}"}]
    })
)

result = json.loads(response['body'].read())
print(result['content'][0]['text'])

Here is the JSON structured data you requested for the scanned historical book:

```json
{
  "title_page": {
    "title": "Mission to Ashantee",
    "author": "T. Edward Bowdich",
    "year": "1819",
    "publisher": ""
  },
  "toc": [
    "chapter-01-geography",
    "chapter-02" 
  ],
  "pages": [
    {
      "page_number": "161",
      "content": "The impression of the Natives that we came \"to spy the country\" was sedulously strengthened by the Moors, who were actuated by alarm, jealousy, and a spirit of intolerance unmitigated by a previous intercourse with Europeans...",
      "chapter": "chapter-01-geography",
      "filename": "chapter-01-geography.qmd",
      "image_path": ""
    },
    {
      "page_number": "162",
      "content": "The difficulty of adjusting geography by investigation only, is not diminished by the numerous small states, scarcely less frequent than those of modern Italy, which we find to compose this part of Western Africa...",
      "chapter": "chapter-01-